# Creating a Basic Blockchain with Python

In [1]:
import hashlib
from datetime import datetime

## Part 1: A Simple Blockchain

#### Remember:
- The blockchain is meant to add new blocks
- It is not meant to change existing blocks
- It is not meant to delete existing blocks

In [ ]:
class Block:
    def __init__(self, timestamp, data, previous_hash = ''):
        # tells us when the block was created
        self.timestamp = timestamp
        # data can be anything .. for crypto currency we might store
        # details of a transaction such as how much money was transferred 
        # who the sender is and who the receiver is!
        self.data = str(data)
        # the hash of the previous block
        # imporant for the integrity of the blockchain
        self.previous_hash = previous_hash
        # the hash of the current block
        self.blockhash = self.calculate_hash()
        
    def calculate_hash(self):
        # calculates the hash of the current block
        # takes the properties of this block and creates a hash for them!
        concat = self.previous_hash + self.timestamp + self.data 
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    def get_block_data(self):
        # returns the properties of current block
        return {
            'timestamp': self.timestamp,
            'data': self.data,
            'previous_hash': self.previous_hash,
            'hash': self.blockhash
        }
   

class BlockChain:
    def __init__(self):
        # the blockchain is going to be a linked list!
        self.chain = []
        self.chain.append(self.create_genesis_block())

    def create_genesis_block(self):
        # creates the very first block in the blockchain
        return Block(get_timestamp(), ["Genesis Block"], "0")
    
    def get_latest_block(self):
        # returns the very last block in the blockchain
        return self.chain[-1]
    
    def add_block(self, new_block):
        # adds a new block to the blockchain
        # set the previous hash
        new_block.previous_hash = self.get_latest_block().blockhash
        # recalculate the hash of the current block because we've changed its previous_hash
        # every time we change something in the block we need to recalculate its hash
        new_block.blockhash = new_block.calculate_hash()
        # add the block to the blockchain
        self.chain.append(new_block)
        
    def is_chain_valid(self):
        # to verify the integrity of the blockchain
        # we go through all blocks (no need to start from the genesis block)
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            # make sure the hash of the current block is still valid
            if current.blockhash != current.calculate_hash():
                return False
            # make sure the current block is connected to the correct previous block
            if current.previous_hash != previous.blockhash:
                return False
        # if we get here then the blockchain is valid
        return True
            

def get_timestamp():
    # A simple function to return the current timestamp!
    date = datetime.now()
    # convert datetime to timestamp
    timestamp = datetime.timestamp(date)
    return str(timestamp)



In [ ]:
bc = BlockChain()
d = {
    'Name': 'Noureddin Sadawi',
    "Nationality": "Libyan",
    "Height": 179,
    "Weight": 80
}
bc.add_block(Block(get_timestamp(), d))

d = {
    'Name': 'Josef Baker',
    "Nationality": "British",
    "Height": 185,
    "Weight": 85
}
bc.add_block(Block(get_timestamp(), d))

In [ ]:
bc.is_chain_valid()

In [ ]:
bc.chain[1].blockhash

In [ ]:
bc.chain[1].data = "Test"
# even if you recalculate the hash of this block, the blockchain is still invalid
# because the relationship with the previous block is now broken. 
bc.chain[1].blockhash = bc.chain[1].calculate_hash()

In [ ]:
bc.is_chain_valid()

In [ ]:
bc.chain[1].blockhash

## Part 2: Proof of Work



#### Extension:
- Now we can create a transaction, compute its hash and add it to the chain  
- Computers can do that very quickly but we don't want that!
- We don't want large numbers of blocks added to the blockchain in a short period of time
- Also, currently we can change a block and recalculate the hashes of the blocks after it and the blockchain will still be valid
- We don't want that!
- To avoid that, blockchains have something called Proof of Work!
- With this mechanism you have to prove that you have put in a lot of computing power into making a Block
- This is called Mining
- BitCoin, for example, requires the hash of a block to begin with a certain number of 0's

- Because we can't influence the output of a hash function, we must try a large number of times and hope to generate a hash with the required number of leading 0's
- This requires a lot of computing power
- It is known as the difficulty level
- It helps in controlling the number of created valid blocks in a certain period of time
- Bitcoin aims for a new block every 10 minutes
- As computers get faster over time, they'll require less time to mine a new block
- To compensate for that, the difficulty level is increased



In [ ]:
class Block:
    def __init__(self, timestamp, data, previous_hash = ''):
        # tells us when the block was created
        self.timestamp = timestamp
        # data can be anything .. for crypto currency we might store
        # details of a transaction such as how much money was transferred 
        # who the sender is and who the receiver is!
        self.data = str(data)
        # the hash of the previous block
        # imporant for the integrity of the blockchain
        self.previous_hash = previous_hash
        # the nonce is needed for mining blocks
        self.nonce = 0
        # the hash of the current block
        self.blockhash = self.calculate_hash()
        
        
    def calculate_hash(self):
        # calculates the hash of the current block
        # takes the properties of this block and creates a hash for them!
        concat = self.previous_hash + self.timestamp + self.data + str(self.nonce)
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    def mine_block(self, difficulty):
        # here we see the benefit of the nonce, we increment it until we get the right hash
        # this is because the hash can always be the same with the same block contents
        while self.blockhash[:difficulty] != "0" * difficulty:
            self.nonce = self.nonce + 1
            self.blockhash = self.calculate_hash()
    
    def get_block_data(self):
        return {
            'timestamp': self.timestamp,
            'data': self.data,
            'previous_hash': self.previous_hash,
            'hash': self.blockhash,
            'nonce': self.nonce
        }
   

class BlockChain:
    def __init__(self, difficulty=2):
        # the blockchain is going to be a linked list!
        self.chain = []
        self.chain.append(self.create_genesis_block())
        self.difficulty = difficulty

    def create_genesis_block(self):
        # creates the very first block in the blockchain
        return Block(get_timestamp(), ["Genesis Block"], "0")
    
    def get_latest_block(self):
        # returns the very last block in the blockchain
        return self.chain[-1]
    
    def add_block(self, new_block):
        # adds a new block to the blockchain
        # set the previous hash
        new_block.previous_hash = self.get_latest_block().blockhash
        # now we call the mine_block method
        new_block.mine_block(self.difficulty)
        # once the block is mined it is added to the blockchain
        self.chain.append(new_block)
    
        
    def is_chain_valid(self):
        # to verify the integrity of the blockchain
        # we go through all blocks (no need to start from the genesis block)
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            # make sure the hash of the current block is still valid
            if current.blockhash != current.calculate_hash():
                return False
            # make sure the current block is connected to the correct previous block
            if current.previous_hash != previous.blockhash:
                return False
        # if we get here then the blockchain is valid
        return True
            

def get_timestamp():
    # A simple function to return the current timestamp!
    date = datetime.now()
    # convert datetime to timestamp
    timestamp = datetime.timestamp(date)
    return str(timestamp)



In [ ]:
bc = BlockChain(difficulty=5)
d = {
    'Name': 'Noureddin Sadawi',
    "Nationality": "Libyan",
    "Height": 179,
    "Weight": 80
}
bc.add_block(Block(get_timestamp(), d))
d = {
    'Name': 'Josef Baker',
    "Nationality": "British",
    "Height": 185,
    "Weight": 85
}
bc.add_block(Block(get_timestamp(), d))

In [ ]:
bc.chain[2].get_block_data()

In [ ]:
bc.chain[1].blockhash

## Part 3: Transactions and Mining Rewards 
### Transform the Blockchain into a Simple Cryptocurrency¬

Extensions:
- Now a block can contain more than one transaction
- Add rewards for mining
- A place to store pending transactions
- A new method to mine a new block for pending transactions

#### Why do we need pending transactions?
Because we create new blocks in a specific interval (Bitcoin makes sure a new block is created every 10mins)

All transactions that are made in between blocks are temporarily stored in a pending transactions list so it can be included in the next block!

When you start a crypto-currency, you need to have virtual money or coins, you need to introduce them somewhere in the system so that users can get coins. Mining rewards steadily add new coins into the system.

In [2]:
class Transaction:
    def __init__(self, from_address, to_address, amount):
        self.from_address = from_address
        self.to_address = to_address
        self.amount = amount
    
class Block:
    def __init__(self, timestamp, transactions, previous_hash = ''):
        # notice no index because the list of blocks is ordered!
        self.timestamp = timestamp
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = 0
        self.blockhash = self.calculate_hash()
        
    def calculate_hash(self):
        concat = self.previous_hash + self.timestamp + str(self.transactions) + str(self.nonce)
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    def mine_block(self, difficulty):
        # here we see the benefit of the nonce, we increment it until we get the right hash
        # this is because the hash can always be the same with the same block contents
        while self.blockhash[:difficulty] != "0" * difficulty:
            self.nonce = self.nonce + 1
            self.blockhash = self.calculate_hash()
    
    def get_block_data(self):
        return {
            'timestamp': self.timestamp,
            'transactions': self.transactions,
            'previous_hash': self.previous_hash,
            'hash': self.blockhash
        }
   

class BlockChain:
    def __init__(self, difficulty=2):
        self.chain = []
        self.chain.append(self.create_genesis_block())
        self.difficulty = difficulty
        # to store pending transactions
        # we have pending transactions because we need to 
        # mine them at every specific time interval
        self.pending_transactions = []
        # reward for mining a block
        self.mining_reward = 100

    def create_genesis_block(self):
        return Block(get_timestamp(), [Transaction('','',0)], "0")

    
    def get_latest_block(self):
        return self.chain[-1]
    
    #def add_block(self, new_block):
    #    new_block.previous_hash = self.get_latest_block().blockhash
    #    new_block.mine_block(self.difficulty)
    #    #new_block.blockhash = new_block.calculate_hash()
    #    self.chain.append(new_block)
    
    # we replace the add block method with this method
    # this method receives an address 
    # when a miner calls this method it'll pass along its wallet address and say:
    # if I successfully mine this block then send the reward to this address
    def mine_pending_transactions(self, mining_reward_address):
        # we start by creating a new block and pass it all pending transactions
        # in a real crypto currency it's not practical to use all pending_transactions
        # there is soooo many of them
        # you might choose which ones to include
        new_block = Block(get_timestamp(), self.pending_transactions)
        # after creating the block, we mine it!
        new_block.mine_block(self.difficulty)
        print('Block mined!')
        # add the block to the chain
        self.chain.append(new_block)
        # reset pending_transactions and give the reward to the miner
        # no from address because the system give you the reward!
        self.pending_transactions = [
            Transaction('', mining_reward_address, self.mining_reward)
        ]
    
        
    def create_transaction(self, transaction):
        # receives a trasaction and adds it to the list of pending trasactions
        self.pending_transactions.append(transaction)
        
    def get_balance(self, address):
        # This method checks the balance of an address
        # In reality we don't have a balance
        # You need to go thru all transactions that involve your address and 
        # compute the balance
        balance = 0
        for b in self.chain:
            for trans in b.transactions:
                # if you're the from_address then you're transferring to someone else
                if trans.from_address == address:
                    balance = balance - trans.amount
                # if you're the to_address then you're receiving from someone else
                if trans.to_address == address:
                    balance = balance + trans.amount
        return balance
        
    def is_chain_valid(self):
        # to verify the integrity of the blockchain
        # we go through all blocks (no need to start from the genesis block)
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            # make sure the hash of the current block is still valid
            if current.blockhash != current.calculate_hash():
                return False
            # make sure the current block is connected to the correct previous block
            if current.previous_hash != previous.blockhash:
                return False
        # if we get here then the blockchain is valid
        return True
    
    

def get_timestamp():
    # A simple function to return the current timestamp!
    date = datetime.now()
    # convert datetime to timestamp
    timestamp = datetime.timestamp(date)
    return str(timestamp)



In [3]:
bc = BlockChain()

In [4]:
bc.pending_transactions

[]

In [5]:
## in reality address 1 and address 2 would be the public key of someone's wallet
bc.create_transaction(Transaction('address1','address2',100))
bc.create_transaction(Transaction('address2','address1',50))

In [6]:
# After creating the transactions, they will be pending
# We have to start the miner to actually create a block for them and store them in the blockchain
bc.mine_pending_transactions('noureddin\'s-address')

Block mined!


In [7]:
# Check the balance
bc.get_balance('noureddin\'s-address')

0

In [ ]:
# Balance is 0 because in our mining method after a block has been mined, we create a new transaction
# to give you your mining reward and that transaction is added to the pending transactions list
# So the mining reward will only be sent when the next block is mined

In [8]:
# Let's mine another one then!
bc.mine_pending_transactions('noureddin\'s-address')

Block mined!


In [9]:
# Check the balance again!
bc.get_balance('noureddin\'s-address')

100

In [ ]:
# But after the second mining, we get a new reward which 
# is in pending transactions and will be included in the next block that is mine

In [10]:
bc.get_balance('address2')

50

## 4. Signing Transactions

Needed for security
* The problem with the current version is that anyone can make any transaction that they want.
* Effectively you can spend coins that aren't yours.
* To solve that we'll make it mandatory for transactions to be signed with a private and public key.
* That way you can only spend coins in a wallet if you have its private key

In [ ]:
#!pip install ecdsa

In [11]:
import ecdsa
import base64


In [12]:
class Transaction:
    ## Now we need to sign the transaction
    # and we need a method to check if this signature is valid
    ## we're going to sign the hash of the transaction
    def __init__(self, from_address, to_address, amount):
        self.from_address = from_address
        self.to_address = to_address
        self.amount = amount
        self.signature = None
    
    def calculate_hash(self):
        ## we're going to sign the hash of the transaction (not all data)
        ## Creates a SHA256 hash of the transaction
        concat = self.from_address + self.to_address + str(self.amount)
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    
    ## Signs a transaction with the given signingKey (which is an Elliptic keypair
    ## object that contains a private key). The signature is then stored inside the
    ## transaction object and later stored on the blockchain.
    def sign_transaction(self, signing_key):
        ## check if your public key == the from address
        ## You can only spend coins from the wallet you have the private key of!
        ## because the priv key is linked to the public key, that means the from add
        ## in the transaction has to equal your public key
        if signing_key.get_verifying_key().to_string().hex() != self.from_address:
            raise ValueError('Error. You cannot sign transactions for other wallets!')
        
        ## Calculate the hash of this transaction, sign it with the key
        ## and store it inside the transaction object
        trans_hash = self.calculate_hash()
        signature = signing_key.sign(bytes(trans_hash.encode('utf-8')))
        self.signature = signature
        
    ## Checks if the signature is valid (transaction has not been tampered with).
    ## It uses the from_address as the public key.
    def is_valid(self):
        ## check if transaction has been correctly signed
        if self.from_address == None: ## mining reward transaction (from_address is not filled in)
            ## mining reward transactions are not signed but they are valid
            return True
        ## if transaction needs a signature, make sure there is a signature
        ## (from_address is filled in)
        ## You can only send a transaction from the wallet that is linked to your
        ## key. So here we check if the from_address matches your publicKey
        if (self.signature == None) or (len(self.signature) == 0):
            raise ValueError('Error. No signature in this transaction!')
        
        ## check if transaction was signed with the correct key
        ## remember the from_address is a public key
        ## from https://www.py4u.net/discuss/164146
        ## create a verification key from from_address (remember it's a public key)
        
        vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(self.from_address), curve=ecdsa.SECP256k1)
        ## we want to verify that the hash of this block has been signed by this signature
        ##print(self.calculate_hash())
        return vk.verify(self.signature, bytes(self.calculate_hash().encode('utf-8'))) # True or False

class Block:
    def __init__(self, timestamp, transactions, previous_hash = ''):
        self.timestamp = timestamp
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = 0
        self.blockhash = self.calculate_hash()
        
    ## Returns the SHA256 of this block (by processing all the data stored
    ## inside this block)
    def calculate_hash(self):
        concat = self.previous_hash + self.timestamp + str(self.transactions) + str(self.nonce)
        return hashlib.sha224(concat.encode('utf-8')).hexdigest()
    
    ## Starts the mining process on the block. It changes the 'nonce' until the hash
    ## of the block starts with enough zeros (= difficulty)
    def mine_block(self, difficulty):
        # here we see the benefit of the nonce, we increment it until we get the right hash
        # this is because the hash can always be the same with the same block contents
        while self.blockhash[:difficulty] != "0" * difficulty:
            self.nonce = self.nonce + 1
            self.blockhash = self.calculate_hash()
    
    ## Get details of the this block
    def get_block_data(self):
        return {
            'timestamp': self.timestamp,
            'transactions': self.transactions,
            'previous_hash': self.previous_hash,
            'hash': self.blockhash
        }
    
    ## Validates all the transactions inside this block (signature + hash) and
    ## returns true if everything checks out. False if the block is invalid.
    def has_valid_transactions(self):
        ## verify all all transactions in the current block
        ## iteratue thru all transactions in the current block
        for trans in self.transactions:
            if not trans.is_valid():
                return False
        return True
    

class BlockChain:
    def __init__(self, difficulty=2):
        self.chain = []
        self.chain.append(self.create_genesis_block())
        self.difficulty = difficulty
        self.pending_transactions = []
        self.mining_reward = 100

    ## Create the very first block on the chain
    def create_genesis_block(self):
        return Block(get_timestamp(), [Transaction('','',0)], "0")

    ## Returns the latest block on our chain. Useful when you want to create a
    ## new Block and you need the hash of the previous Block.
    def get_latest_block(self):
        return self.chain[-1]
    
    #def add_block(self, new_block):
    #    new_block.previous_hash = self.get_latest_block().blockhash
    #    new_block.mine_block(self.difficulty)
    #    #new_block.blockhash = new_block.calculate_hash()
    #    self.chain.append(new_block)
    
    
    ## Takes all the pending transactions, puts them in a Block and starts the
    ## mining process. It also adds a transaction to send the mining reward to
    ## the given address.
    def mine_pending_transactions(self, mining_reward_address):
        # in a real crypto currency it's not practical to use all pending_transactions
        # there is soooo many of them
        # you might choose which ones to include
        
        # reset pending_transactions
        self.pending_transactions.append(
            Transaction(None, mining_reward_address, self.mining_reward)
        )
        # create a new block by adding the pending_transactions and the hash of the latest block
        new_block = Block(get_timestamp(), self.pending_transactions, self.get_latest_block().blockhash)
        new_block.mine_block(self.difficulty)
        # print('Block mined!')
        self.chain.append(new_block)
        self.pending_transactions = []
        
    
    ## Add a new transaction to the list of pending transactions (to be added
    ## next time the mining process starts). This verifies that the given
    ## transaction is properly signed.
    def add_transaction(self, transaction):
        # we're not creating a transaction, we're receiving it and adding it to the pending_transactions list
        if (transaction.from_address == None) or (transaction.to_address == None):
            raise ValueError('Error. Transaction must have from and to addresses!')
        # make sure transaction is valid (verify it)
        if not transaction.is_valid():
            raise ValueError('Error. Cannot add invalid transaction to the chain!')
        
        ## Making sure that the amount is > 0
        if transaction.amount <= 0:
            raise ValueError('Error. Transaction amount should be greater than 0!')
            
        ## Making sure that the amount sent is not greater than existing balance
        #if self.get_balance(transaction.from_address) < transaction.amount:
        #    print(self.get_balance(transaction.from_address))
        #    print(transaction.amount)
        #    raise ValueError('Error. Not enough balance!')
          
        self.pending_transactions.append(transaction)
        
        
    ## Returns the balance of a given wallet address.    
    def get_balance(self, address):
        balance = 0
        for b in self.chain:
            for trans in b.transactions:
                if trans.from_address == address:
                    balance = balance - trans.amount
                if trans.to_address == address:
                    balance = balance + trans.amount
        return balance
        
    ## Loops over all the blocks in the chain and verify if they are properly
    ## linked together and nobody has tampered with the hashes. By checking
    ## the blocks it also verifies the (signed) transactions inside of them.
    def is_chain_valid(self):
        ## TODO?: Check if the Genesis block hasn't been tampered with by comparing
        ## the output of create_genesis_block with the first block on our chain??
    
        # Goes thru all blocks in the chain and verifies that the hashes are correct
        # and each block links to the previous block
        # we'll also need to verify that all the transactions in the current block are valid
        for i in range(1, len(self.chain)):
            #Check the remaining blocks on the chain to see if there hashes and
            # signatures are correct
            current = self.chain[i]
            previous = self.chain[i-1]
            
            if not current.has_valid_transactions():
                return False
            
            if current.blockhash != current.calculate_hash():
                return False
            if current.previous_hash != previous.calculate_hash():
                return False
        return True
            
## Returns a timestamp of NOW
def get_timestamp():
    date = datetime.now()
    # convert datetime to timestamp
    timestamp = datetime.timestamp(date)
    return str(timestamp)



In [13]:

sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1) #this is your sign key (private key)
private_key = sk.to_string().hex() #convert your private key to hex
vk = sk.get_verifying_key() #this is your verification key (public key)
public_key = vk.to_string().hex()#this is the wallet address

# We need them to sign transactions and to verify our balance (how much money is in our wallet)
print(private_key)
print(public_key)

c450c9bf7d174734b222e9777a80dba9728e1eab26ef3219b5f2df360084ff96
835233605009cf77e3574698990dd08b0b44392eca64570ec8ad3ff1a8b0de53222ef25dcd93a77b05abda2f27ae31a23df3afc1cce39e31ae0574179ea9f142


In [14]:
coin = BlockChain()
# Create a transaction.. from my wallet to someone else's wallet
tx1 = Transaction(public_key, 'someone else\'s public key', 10)
# We have to sign the transaction then add it to the Blockchain
tx1.sign_transaction(sk)
coin.add_transaction(tx1)

# When we mine, we have to specify where the mining reward goes
# If we use an address that does not exist, the coins will be sent 
# to a wallet and nobody will be able to access it because nobody has
# the private key of that wallet
# Let's send the reward to my wallet (by using my public key)
coin.mine_pending_transactions(public_key)

In [15]:
# Why 90?
# Because when we mine a block we get a 100 coins (and we have spent 10 of them)
coin.get_balance(public_key)

90

In [16]:
coin.is_chain_valid()

True

In [17]:
(coin.chain[0].get_block_data())

{'timestamp': '1675625919.196356',
 'transactions': [<__main__.Transaction at 0x1105c39d0>],
 'previous_hash': '0',
 'hash': '84c7ad5431f27ad370f85773b82ad39c1c504b20b860f1aae61151f0'}

In [18]:
public_key

'835233605009cf77e3574698990dd08b0b44392eca64570ec8ad3ff1a8b0de53222ef25dcd93a77b05abda2f27ae31a23df3afc1cce39e31ae0574179ea9f142'